In [1]:
import pandas as pd

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

c:\Users\Usuario\anaconda3\envs\clase\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Limpieza tablas extraidas desde la API PLACE de Google: 

#### Todas las tablas tienen la misma estructura por lo que la misma limpieza se la aplico a todas.

In [2]:
caribe = pd.read_csv('../data/caribe.csv', encoding='latin1')
coreano = pd.read_csv('../data/coreano.csv', encoding='latin1')
india = pd.read_csv('../data/india.csv', encoding='latin1')
ita = pd.read_csv('../data/ita.csv', encoding='latin1')
japo = pd.read_csv('../data/japo.csv', encoding='latin1')
mex = pd.read_csv('../data/mex.csv', encoding='latin1')
peru = pd.read_csv('../data/peru.csv', encoding='latin1')
chino = pd.read_csv('../data/chino.csv', encoding='latin1')

In [3]:
coreano.info() # Aqui veo que solo price-level tiene nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   business_status     20 non-null     object 
 1   geometry            20 non-null     object 
 2   name                20 non-null     object 
 3   photos              20 non-null     object 
 4   price_level         12 non-null     float64
 5   rating              20 non-null     float64
 6   types               20 non-null     object 
 7   user_ratings_total  20 non-null     int64  
 8   tipo                20 non-null     object 
dtypes: float64(2), int64(1), object(6)
memory usage: 1.5+ KB


In [4]:
coreano.columns

Index(['business_status', 'geometry', 'name', 'photos', 'price_level',
       'rating', 'types', 'user_ratings_total', 'tipo'],
      dtype='object')

In [5]:
coreano=coreano.drop(columns=['business_status','geometry','tipo'])

In [6]:
coreano.columns = ['Nombre','Foto','Precio','Puntuacion','Servicios', 'Nº opiniones']

In [7]:
coreano.Nombre = coreano.Nombre.str.upper()

In [8]:
coreano.head() 

,Nombre,Foto,Precio,Puntuacion,Servicios,Nº opiniones
0,RESTAURANTE COREANO GANGNAM,"[{'height': 2988, 'html_attributions': ['<a hr...",1.0,4.4,"['restaurant', 'food', 'point_of_interest', 'e...",597
1,RESTAURANTE KOREA,"[{'height': 2773, 'html_attributions': ['<a hr...",2.0,4.5,"['restaurant', 'food', 'point_of_interest', 'e...",1922
2,GO HYANG MAT,"[{'height': 3000, 'html_attributions': ['<a hr...",1.0,4.5,"['restaurant', 'food', 'point_of_interest', 'e...",994
3,RESTAURANTE MIDANG,"[{'height': 1669, 'html_attributions': ['<a hr...",2.0,4.7,"['meal_delivery', 'restaurant', 'food', 'point...",762
4,RESTAURANTE MARU,"[{'height': 3024, 'html_attributions': ['<a hr...",2.0,4.3,"['restaurant', 'food', 'point_of_interest', 'e...",847


#### Creo una función para limpiar 🫧🧹 todo de una.

In [9]:
import sys
sys.path.append("..")
from src.funciones import *  #funciones

In [10]:
india = clean(india) 
ita = clean(ita) 
japo = clean(japo)
mex = clean(mex) 
peru = clean(peru)
chino = clean(chino)

#### La talba de Caribe tiene una columna menos por lo que la funcion no se puede aplicar

In [11]:
caribe=caribe.drop(columns=['business_status','geometry','tipo'])
caribe.columns = ['Nombre','Foto','Puntuacion','Servicios', 'Nº opiniones']
caribe.Nombre = caribe.Nombre.str.upper()

#### Nuesto principal objetivo 🎯🎯 es tener todos los nombres de los restaurantes de las tablas para poder relacionarlos con la tabla de locales

In [12]:
locales = pd.read_csv('../data/restaurantes.csv', encoding='latin1')

In [13]:
locales.columns = ['o', 'id_local','Nombre']

In [14]:
df_list = [caribe, japo, chino, mex, peru, india, ita]


results = []

for df in df_list:
    result = locales.merge(df, on='Nombre', how='inner')
    results.append(result)

# Concatenar todos los resultados en un solo DataFrame
result = pd.concat(results, ignore_index=True)

In [15]:
result = result.drop_duplicates(subset=['id_local'], keep=False)


In [16]:
result= result.drop(columns=['o','Foto'])

In [17]:
result.head()

,id_local,Nombre,Puntuacion,Servicios,Nº opiniones,Precio
0,17202,KABUKI,4.6,"['restaurant', 'food', 'point_of_interest', 'e...",876,4.0
1,50673,KABUKI,4.6,"['restaurant', 'food', 'point_of_interest', 'e...",876,4.0
2,33122,HATTORI HANZO,3.9,"['restaurant', 'food', 'point_of_interest', 'e...",3484,2.0
3,78750,PILAR AKANEYA,4.8,"['restaurant', 'food', 'point_of_interest', 'e...",1188,NaN
4,1792,RESTAURANTE CHINO GRAN SIGLO,4.3,"['restaurant', 'food', 'point_of_interest', 'e...",636,1.0


In [18]:
result.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, 0 to 53
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_local      24 non-null     int64  
 1   Nombre        24 non-null     object 
 2   Puntuacion    24 non-null     float64
 3   Servicios     24 non-null     object 
 4   Nº opiniones  24 non-null     int64  
 5   Precio        22 non-null     float64
dtypes: float64(2), int64(2), object(2)
memory usage: 5.4 KB


In [19]:
optim(result)
result.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, 0 to 53
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   id_local      24 non-null     int32   
 1   Nombre        24 non-null     category
 2   Puntuacion    24 non-null     float32 
 3   Servicios     24 non-null     category
 4   Nº opiniones  24 non-null     int16   
 5   Precio        22 non-null     float32 
dtypes: category(2), float32(2), int16(1), int32(1)
memory usage: 3.3 KB


In [20]:
result.to_csv('../data/Opiniones.csv',index=False)

In [21]:
india = optim(india) 
ita = optim(ita) 
japo = optim(japo)
mex = optim(mex) 
peru = optim(peru)
chino = optim(chino)
caribe = optim(caribe)

In [22]:
india.to_csv('../data/Indio.csv',index=False)
ita.to_csv('../data/Italiano.csv',index=False)
japo.to_csv('../data/Japonés.csv',index=False)
mex.to_csv('../data/Mexicano.csv',index=False)
peru.to_csv('../data/Peruano.csv',index=False)
chino.to_csv('../data/Chino.csv',index=False)
caribe.to_csv('../data/Carie.csv',index=False)